In [1]:
import os
import pandas as pd


import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import power_transform
from catboost import CatBoostRegressor

# Local dependencies
from src.constants import (
    PREPROCESSING_PATH,
    TARGET
)
from utils.classes.extractor import FunctionalGroupToGramExtractor

from typing import (
    Optional,
    List,
    Any
)
from sklearn.model_selection import KFold, RepeatedKFold, RepeatedStratifiedKFold

import pickle
import json

import warnings
warnings.filterwarnings("ignore")

In [2]:
Train = pd.read_csv(r"C:\Users\szjt\Downloads\tmlcc-2021\tmlcc-2021\Train_Prem3.csv")
Train = Train.set_index(Train.columns[0])

Pretest = pd.read_csv(f"test_Prem.csv")
Pretest = Pretest.set_index(Pretest.columns[0])

funtional_group_extractor = FunctionalGroupToGramExtractor()

df_test_rdf = pd.read_csv(r"C:\Users\szjt\Downloads\tmlcc-2021\tmlcc-2021\test_rdf_encoded_df_v1.csv")
df_rdf = pd.read_csv(r"C:\Users\szjt\Downloads\tmlcc-2021\tmlcc-2021\train_rdf_encoded_df_v1.csv")


In [3]:
#Add rdf columns
#'''
#Train = pd.concat([Train, df['MOFname']], axis = 1)
#Pretest = pd.concat([Pretest, Pretest['MOFname']], axis = 1)

#df_rdf['Structure_Name'] =df_rdf['Structure_Name'].str.replace(".cif","")
#df_test_rdf['Structure_Name'] =df_test_rdf['Structure_Name'].str.replace(".cif","")

Train = Train.merge(df_rdf, left_on = 'MOFname', right_on = 'MOFname')
Pretest = Pretest.merge(df_test_rdf, left_on = 'MOFname', right_on = 'MOFname')

#Train = Train.drop(['Structure_Name'], axis = 1)
#Pretest = Pretest.drop(['Structure_Name'], axis = 1)
#'''


In [4]:
Pretest = Pretest.set_index("MOFname")
Pretest.head()

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,topology,...,transfomers_encode_329,transfomers_encode_330,transfomers_encode_331,transfomers_encode_332,transfomers_encode_333,transfomers_encode_334,transfomers_encode_335,transfomers_encode_336,transfomers_encode_337,transfomers_encode_338
MOFname,,,,,,,,,,,,,,,,,,,,,
mof_unit_68614,1208.301332,797.70936,586.54,0.11392,0.1039,OEt,2,5,26,pcu,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mof_unit_68615,4126.414623,3733.65779,852.49,0.21367,0.1422,H-I,4,6,17,acs,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mof_unit_68616,1602.148373,747.21048,3155.73,0.33883,0.4375,CN-OH,3,11,17,pcu,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mof_unit_68617,2436.629312,995.80232,3521.09,0.40464,0.5963,OMe,2,1,28,pcu,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mof_unit_68618,3123.418006,1337.53800,2678.46,0.38959,0.5479,NO2-Pr,3,8,19,pcu,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_linker_correction = pd.read_csv(r"C:\Users\szjt\Downloads\tmlcc-2021\tmlcc-2021\sra_2to15.csv")
df_linker_correction

,MOFname,metal_linker,organic_linker1,organic_linker2,topology,flag
0,mof_unit_43,9,15,25,sra,train
1,mof_unit_51,9,12,15,sra,train
2,mof_unit_69,9,15,2,sra,train
3,mof_unit_100,9,12,15,sra,train
4,mof_unit_165,9,11,15,sra,train
...,...,...,...,...,...,...
1243,mof_unit_82650,9,11,15,sra,test
1244,mof_unit_82679,9,11,15,sra,test
1245,mof_unit_82843,9,4,15,sra,test
1246,mof_unit_82874,9,15,25,sra,test


In [6]:
Train = Train.set_index('MOFname')
df_linker_correction = df_linker_correction.set_index('MOFname')
Train.update(df_linker_correction)
Train.reset_index(inplace=True)
Pretest.update(df_linker_correction)
Pretest.reset_index(inplace=True)

In [7]:
Train.iloc[42:51]

,MOFname,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],functional_groups,metal_linker,organic_linker1,organic_linker2,...,transfomers_encode_329,transfomers_encode_330,transfomers_encode_331,transfomers_encode_332,transfomers_encode_333,transfomers_encode_334,transfomers_encode_335,transfomers_encode_336,transfomers_encode_337,transfomers_encode_338
42,mof_unit_43,2379.449687,1423.61040,1140.40,0.18193,0.1831,NO2-OH,9.0,15.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,mof_unit_44,995.320211,591.51620,1224.26,0.19587,0.1985,Ph,2.0,10.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,mof_unit_45,2071.222251,1364.69264,1259.37,0.20088,0.1836,OMe,4.0,11.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,mof_unit_46,1105.846254,677.56392,1430.01,0.21098,0.2074,OH,2.0,14.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,mof_unit_47,1123.837854,839.68306,696.86,0.13735,0.1107,HCO-SO3H,2.0,4.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,mof_unit_48,5864.950079,1745.95526,4262.41,0.56340,1.1397,Cl-HCO,12.0,8.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,mof_unit_49,3094.394961,1712.88556,1457.73,0.23200,0.2524,COOH-OEt,9.0,16.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,mof_unit_50,1650.014869,897.54908,0.00,0.16651,0.1843,OPr,3.0,4.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,mof_unit_51,1640.841092,1192.58024,0.00,0.04781,0.0396,NHMe,9.0,12.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
metal_linker = pd.read_csv("metal_linker_agg.csv", index_col=0)
metal_linker = metal_linker[[
    "metal_linker__convex_hull_area",
    "metal_linker__convex_hull_volume",
    "metal_linker__convex_hull_npoints",
    "metal_linker__convex_hull_nsimplex"
]]

In [9]:
class PreprocessingPipeline:
    funcgroup2num = None
    topology2num = None
    spacegroup2num = None
    organic_linker1 = None
    organic_linker2 = None

    def label(self, df):
        # if self.funcgroup2num is None:
        #     self.funcgroup2num = {
        #         c: i
        #         for i, c
        #         in enumerate(df["functional_groups"].unique())
        #     }

        # if self.topology2num is None:
        #     self.topology2num = {
        #         c: i
        #         for i, c
        #         in enumerate(df["topology"].unique())
        #     }
        # df["label_topology"] = df["topology"].map(self.topology2num)

        df["label_topology_pcu"] = (df["topology"] == "pcu").astype(int)
        df["label_topology_sra"] = (df["topology"] == "sra").astype(int)
        df["label_topology_acs"] = (df["topology"] == "acs").astype(int)
        df["label_topology_etb"] = (df["topology"] == "etb").astype(int)
        df["label_topology_bcu"] = (df["topology"] == "bcu").astype(int)
        df["label_topology_nbo"] = (df["topology"] == "nbo").astype(int)
        
        
        df["label_spacegroup_triclinic"] = (
            df["_space_group_crystal_system"] == "triclinic"
        ).astype(int)

        df["metal_linker_1"] = (
            df["metal_linker"] == 1
        ).astype(int)
        df["metal_linker_2"] = (
            df["metal_linker"] == 2
        ).astype(int)
        df["metal_linker_3"] = (
            df["metal_linker"] == 3
        ).astype(int)
        df["metal_linker_4"] = (
            df["metal_linker"] == 4
        ).astype(int)
        df["metal_linker_9"] = (
            df["metal_linker"] == 9
        ).astype(int)
        df["metal_linker_10"] = (
            df["metal_linker"] == 10
        ).astype(int)
        df["metal_linker_12"] = (
            df["metal_linker"] == 12
        ).astype(int)

        if self.organic_linker1 is None:
            self.organic_linker1 = set(df['organic_linker1'])
        for linker in self.organic_linker1:
            if linker == 59:
                continue
            df[f"organic_linker1_{linker}"] = (
                df["organic_linker1"] == linker
            ).astype(int)

        if self.organic_linker2 is None:
            self.organic_linker2 = set(df['organic_linker2'])
        for linker in self.organic_linker2:
            if linker == 59:
                continue
            df[f"organic_linker2_{linker}"] = (
                df["organic_linker2"] == linker
            ).astype(int)

        return df
    

    @staticmethod
    def get_density(df: pd.DataFrame):
        df['density'] = (df["weight [u]"] / df["volume [A^3]"]) * 1.66054
        df['void_volume [cm^3/g]'] = df['void_fraction'] / df["density"]
        return df

    @staticmethod
    def replace_surface_area_equal_0_with_null(df: pd.DataFrame):
        df["surface_area [m^2/g]"] = df["surface_area [m^2/g]"].replace(0, np.nan)
        return df

    @staticmethod
    def replace_inf_with_null(df: pd.DataFrame):
        df = df.replace(np.inf, 999999)
        df = df.replace(-np.inf, -999999)
        return df

    @staticmethod
    def drop_unused_columns(
        df, unused_columns: Optional[List[str]] = None
    ) -> pd.DataFrame:
        if not unused_columns:
            unused_columns = [
                "MOFname",
                # 'functional_groups',
                #"topology",
                "cif_filepath",
                "_audit_creation_date",
                "_symmetry_Int_Tables_number",
                "_symmetry_space_group_name_H-M",
                #"_space_group_crystal_system",
                #'metal_linker',
#                 'organic_linker1',
#                 'organic_linker2'
                "partial_charge_mean",
                "partial_charge_std",
                "_cell_volume",
#                 "bond_type_countAm",
#                 'mof_convex_hull_volume',
#                 'mof_convex_hull_area'
#                 'mof_convex_hull_npoints',
#                 'mof_convex_hull_nsimple',
#                 'bond_type_countT'
            ]
        for col in unused_columns:
            try:
                df.drop(col, axis=1, inplace=True)
            except KeyError:
                pass
        return df

    @staticmethod
    def set_imputer(X: pd.DataFrame) -> KNNImputer:
        imputer = KNNImputer(n_neighbors=5)
        imputer.fit(X)
        return imputer

    @staticmethod
    def impute_value(X: pd.DataFrame, imputer: KNNImputer):
        return imputer.transform(X)

    @staticmethod
    def extract_functional_group(X: pd.DataFrame, fit: bool) -> pd.DataFrame:
        funtional_group_extacted = funtional_group_extractor.transform(X, fit)
        return pd.concat([X, funtional_group_extractor], axis=1)


class TrainDataPreprocessingPipeline(PreprocessingPipeline):
    @staticmethod
    def drop_surface_area_equal_minus_1(df):
        return df.drop(df[df["surface_area [m^2/g]"] == -1].index)
    
    @staticmethod
    def drop_surface_area_equal_0(df):
        return df.drop(df[df["surface_area [m^2/g]"] == 0].index)
    
    @staticmethod
    def drop_CO2_N2_selectivity_equal_0(df):
        return df.drop(df[df["CO2/N2_selectivity"] == 0].index)
    
    
    @staticmethod
    def drop_heat_adsorption(df):
        df = df.drop(df[df["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] == 0].index)
        df = df.drop(df[df["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] == np.inf].index)
        df = df.drop(df[df["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"] == -np.inf].index)
        df = df.drop(df[df["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"].isna()].index)
        return df

    def impute(self):
        temp_columns = self.X.columns
        temp_index = self.X.index

        if not self.imputer:
            self.imputer = self.set_imputer(self.X)

        self.X = self.impute_value(self.X, self.imputer)

        self.X = pd.DataFrame(self.X)
        self.X.columns = temp_columns
        self.X.index = temp_index

    def run(self):
        self.functional_group_extractor = FunctionalGroupToGramExtractor()

        # Drop and add featrures
        print("Print droping and replace null")
        self.df = self.drop_surface_area_equal_minus_1(self.df)
        self.df = self.drop_surface_area_equal_0(self.df)
        self.df = self.replace_surface_area_equal_0_with_null(self.df)
        self.df = self.drop_heat_adsorption(self.df)
        
        self.df = self.replace_inf_with_null(self.df)
        self.df = self.label(self.df)
        self.df = self.drop_unused_columns(self.df)
        self.df = self.df.drop(self.df[self.df["CO2/N2_selectivity"] == 0].index)
        self.df = self.get_density(self.df)
        # Split
        print("Split X, y")
        self.X = self.df.drop(TARGET, axis=1)
        self.y = self.df[[TARGET]]

        # Extract
        print("Extract functional group")
        functional_group = self.functional_group_extractor.transform(
            self.X[['functional_groups']],
            fit=True
        )
        
        # Impute
        print("Impute")
        self.X = self.X.drop("functional_groups", axis=1)
        self.impute()
        self.X = pd.concat([self.X, functional_group], axis=1)
        assert not (np.any(self.X.isin([-np.inf, np.inf, np.nan])))

    def __init__(self, df: pd.DataFrame, imputer: Any = None):
        self.df = df
        self.imputer = imputer


In [10]:
class TestDataPreprocessingPipeline(PreprocessingPipeline):
    def impute(self):
        temp_columns = self.X.columns
        temp_index = self.X.index

        if not self.imputer:
            self.imputer = self.set_imputer(self.X)

        self.X = self.impute_value(self.X, self.imputer)

        self.X = pd.DataFrame(self.X)
        self.X.columns = temp_columns
        self.X.index = temp_index

    def run(self):
        # Drop
        print("Print droping and replace null")
        self.X = self.replace_surface_area_equal_0_with_null(self.df)
        self.X = self.replace_inf_with_null(self.X)
        self.X = self.label(self.X)
        self.X = self.drop_unused_columns(self.X)
        self.X = self.get_density(self.X)

        # Extract
        print("Extract functional group")
        functional_group = self.functional_group_extractor.transform(
            self.X[['functional_groups']],
        )

        # Impute
        print("Impute")
        self.X = self.X.drop('functional_groups', axis=1)
        self.impute()
        self.X = pd.concat([self.X, functional_group], axis=1)
        self.X = self.X[self.columns]

    
    def __init__(
        self,
        df: pd.DataFrame,
        imputer: Any = None,
        functional_group_extractor: FunctionalGroupToGramExtractor = None,
        columns: list = None
    ):
        self.df = df
        self.imputer = imputer
        self.functional_group_extractor = functional_group_extractor
        self.columns = columns



In [11]:
class Imputer9999:
    def transform(self, df):
        return df.replace(np.nan, -9999)

imputer = Imputer9999()
train = TrainDataPreprocessingPipeline(Train, imputer)
train.run()

Print droping and replace null
Split X, y
Extract functional group
Impute


In [12]:
test = TestDataPreprocessingPipeline(
    Pretest,
    train.imputer,
    train.functional_group_extractor,
    train.X.columns
)
test.organic_linker1 = train.organic_linker1
test.organic_linker2 = train.organic_linker2
test.topology2num = train.topology2num
test.spacegroup2num = train.spacegroup2num
test.run()

Print droping and replace null
Extract functional group
Impute


In [13]:
train.X

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,...,functional_group_bigram_Et-O,functional_group_bigram_O-Et,functional_group_bigram_H-F,functional_group_bigram_O-Pr,functional_group_bigram_O2-O,functional_group_bigram_H-H,functional_group_bigram_C-N,functional_group_bigram_Me-H,functional_group_bigram_H-N,functional_group_bigram_Pr-O
1,2769.503842,2211.697211,603.61,0.13794,0.104020,10.0,44.0,57.0,etb,33.616780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1089.818728,773.687960,788.50,0.14874,0.126173,2.0,22.0,24.0,pcu,19.263726,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2205.198301,1304.638720,1441.53,0.21814,0.222046,9.0,17.0,24.0,sra,25.701377,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3954.659761,1543.027680,2430.55,0.37094,0.572519,9.0,7.0,23.0,sra,17.146541,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3565.914939,1954.749656,1530.02,0.33337,0.366233,10.0,53.0,55.0,etb,18.363791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66519,1426.479810,1272.451540,1343.62,0.30190,0.203816,3.0,12.0,21.0,pcu,5.867674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66520,23943.701366,5497.752320,4182.24,0.66340,1.739932,1.0,9.0,27.0,pcu,4.060772,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66521,14389.971556,4396.164320,4149.64,0.57051,1.124605,1.0,9.0,20.0,pcu,4.313411,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
66522,16997.806645,3932.703680,4326.62,0.66963,1.742959,2.0,2.0,5.0,nbo,3.447440,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [14]:
train.y

,CO2_working_capacity [mL/g]
1,101.224774
2,118.987011
3,187.626004
5,55.786959
6,111.690462
...,...
66519,7.602105
66520,2.675231
66521,-1.686092
66522,-7.546805


In [15]:
test.X

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,...,functional_group_bigram_Et-O,functional_group_bigram_O-Et,functional_group_bigram_H-F,functional_group_bigram_O-Pr,functional_group_bigram_O2-O,functional_group_bigram_H-H,functional_group_bigram_C-N,functional_group_bigram_Me-H,functional_group_bigram_H-N,functional_group_bigram_Pr-O
0,1208.301332,797.70936,586.54,0.11392,0.103916,2.0,5.0,26.0,pcu,36.639791,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4126.414623,3733.65779,852.49,0.21367,0.142211,4.0,6.0,17.0,acs,18.390691,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1602.148373,747.21048,3155.73,0.33883,0.437514,3.0,11.0,17.0,pcu,13.062850,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2436.629312,995.80232,3521.09,0.40464,0.596260,2.0,1.0,28.0,pcu,9.601198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3123.418006,1337.53800,2678.46,0.38959,0.547876,3.0,8.0,19.0,pcu,12.974954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,32660.944605,4723.68288,5720.14,0.77614,3.231757,3.0,3.0,14.0,nbo,4.536626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16996,5070.998617,1499.13262,4017.28,0.59192,1.205777,3.0,1.0,11.0,nbo,6.745508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16997,4669.804446,1322.04892,4288.76,0.54950,1.168879,2.0,7.0,23.0,pcu,4.666206,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16998,4682.120862,1213.51148,4331.86,0.60643,1.409062,3.0,7.0,25.0,pcu,4.823305,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)

In [17]:
train.X.head()

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],_space_group_crystal_system,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,mof_convex_hull_area,mof_convex_hull_volume,mof_convex_hull_npoints,mof_convex_hull_nsimplex,partial_charge_median,bond_type_countS,bond_type_countA,bond_type_countD,bond_type_countT,bond_type_countAm,transfomers_encode_0,transfomers_encode_1,transfomers_encode_2,transfomers_encode_3,transfomers_encode_4,transfomers_encode_5,transfomers_encode_6,transfomers_encode_7,transfomers_encode_8,transfomers_encode_9,transfomers_encode_10,transfomers_encode_11,transfomers_encode_12,transfomers_encode_13,transfomers_encode_14,transfomers_encode_15,transfomers_encode_16,transfomers_encode_17,transfomers_encode_18,transfomers_encode_19,transfomers_encode_20,transfomers_encode_21,transfomers_encode_22,transfomers_encode_23,transfomers_encode_24,transfomers_encode_25,transfomers_encode_26,transfomers_encode_27,transfomers_encode_28,transfomers_encode_29,transfomers_encode_30,transfomers_encode_31,transfomers_encode_32,transfomers_encode_33,transfomers_encode_34,transfomers_encode_35,transfomers_encode_36,transfomers_encode_37,transfomers_encode_38,transfomers_encode_39,transfomers_encode_40,transfomers_encode_41,transfomers_encode_42,transfomers_encode_43,transfomers_encode_44,transfomers_encode_45,transfomers_encode_46,transfomers_encode_47,transfomers_encode_48,transfomers_encode_49,transfomers_encode_50,transfomers_encode_51,transfomers_encode_52,transfomers_encode_53,transfomers_encode_54,transfomers_encode_55,transfomers_encode_56,transfomers_encode_57,transfomers_encode_58,transfomers_encode_59,transfomers_encode_60,transfomers_encode_61,transfomers_encode_62,transfomers_encode_63,transfomers_encode_64,transfomers_encode_65,transfomers_encode_66,transfomers_encode_67,transfomers_encode_68,transfomers_encode_69,transfomers_encode_70,transfomers_encode_71,transfomers_encode_72,transfomers_encode_73,transfomers_encode_74,transfomers_encode_75,transfomers_encode_76,transfomers_encode_77,transfomers_encode_78,transfomers_encode_79,transfomers_encode_80,transfomers_encode_81,transfomers_encode_82,transfomers_encode_83,transfomers_encode_84,transfomers_encode_85,transfomers_encode_86,transfomers_encode_87,transfomers_encode_88,transfomers_encode_89,transfomers_encode_90,transfomers_encode_91,transfomers_encode_92,transfomers_encode_93,transfomers_encode_94,transfomers_encode_95,transfomers_encode_96,transfomers_encode_97,transfomers_encode_98,transfomers_encode_99,transfomers_encode_100,transfomers_encode_101,transfomers_encode_102,transfomers_encode_103,transfomers_encode_104,transfomers_encode_105,transfomers_encode_106,transfomers_encode_107,transfomers_encode_108,transfomers_encode_109,transfomers_encode_110,transfomers_encode_111,transfomers_encode_112,transfomers_encode_113,transfomers_encode_114,transfomers_encode_115,transfomers_encode_116,transfomers_encode_117,transfomers_encode_118,transfomers_encode_119,transfomers_encode_120,transfomers_encode_121,transfomers_encode_122,transfomers_encode_123,transfomers_encode_124,transfomers_encode_125,transfomers_encode_126,transfomers_encode_127,transfomers_encode_128,transfomers_encode_129,transfomers_encode_130,transfomers_encode_131,transfomers_encode_132,transfomers_encode_133,transfomers_encode_134,transfomers_encode_135,transfomers_encode_136,transfomers_encode_137,transfomers_encode_138,transfomers_encode_139,transfomers_encode_140,transfomers_encode_141,transfomers_encode_142,transfomers_encode_143,transfomers_encode_144,transfomers_encode_145,transfomers_encode_146,transfomers_encode_147,transfomers_encode_148,transfomers_encode_149,transfomers_encode_150,transfomers_encode_151,transfomers_encode_152,transfomers_encode_153,transfomers_encode_154,transfomers_encode

In [18]:
df = pd.concat([train.X, train.y], axis =1)
df.head()

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],_space_group_crystal_system,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,mof_convex_hull_area,mof_convex_hull_volume,mof_convex_hull_npoints,mof_convex_hull_nsimplex,partial_charge_median,bond_type_countS,bond_type_countA,bond_type_countD,bond_type_countT,bond_type_countAm,transfomers_encode_0,transfomers_encode_1,transfomers_encode_2,transfomers_encode_3,transfomers_encode_4,transfomers_encode_5,transfomers_encode_6,transfomers_encode_7,transfomers_encode_8,transfomers_encode_9,transfomers_encode_10,transfomers_encode_11,transfomers_encode_12,transfomers_encode_13,transfomers_encode_14,transfomers_encode_15,transfomers_encode_16,transfomers_encode_17,transfomers_encode_18,transfomers_encode_19,transfomers_encode_20,transfomers_encode_21,transfomers_encode_22,transfomers_encode_23,transfomers_encode_24,transfomers_encode_25,transfomers_encode_26,transfomers_encode_27,transfomers_encode_28,transfomers_encode_29,transfomers_encode_30,transfomers_encode_31,transfomers_encode_32,transfomers_encode_33,transfomers_encode_34,transfomers_encode_35,transfomers_encode_36,transfomers_encode_37,transfomers_encode_38,transfomers_encode_39,transfomers_encode_40,transfomers_encode_41,transfomers_encode_42,transfomers_encode_43,transfomers_encode_44,transfomers_encode_45,transfomers_encode_46,transfomers_encode_47,transfomers_encode_48,transfomers_encode_49,transfomers_encode_50,transfomers_encode_51,transfomers_encode_52,transfomers_encode_53,transfomers_encode_54,transfomers_encode_55,transfomers_encode_56,transfomers_encode_57,transfomers_encode_58,transfomers_encode_59,transfomers_encode_60,transfomers_encode_61,transfomers_encode_62,transfomers_encode_63,transfomers_encode_64,transfomers_encode_65,transfomers_encode_66,transfomers_encode_67,transfomers_encode_68,transfomers_encode_69,transfomers_encode_70,transfomers_encode_71,transfomers_encode_72,transfomers_encode_73,transfomers_encode_74,transfomers_encode_75,transfomers_encode_76,transfomers_encode_77,transfomers_encode_78,transfomers_encode_79,transfomers_encode_80,transfomers_encode_81,transfomers_encode_82,transfomers_encode_83,transfomers_encode_84,transfomers_encode_85,transfomers_encode_86,transfomers_encode_87,transfomers_encode_88,transfomers_encode_89,transfomers_encode_90,transfomers_encode_91,transfomers_encode_92,transfomers_encode_93,transfomers_encode_94,transfomers_encode_95,transfomers_encode_96,transfomers_encode_97,transfomers_encode_98,transfomers_encode_99,transfomers_encode_100,transfomers_encode_101,transfomers_encode_102,transfomers_encode_103,transfomers_encode_104,transfomers_encode_105,transfomers_encode_106,transfomers_encode_107,transfomers_encode_108,transfomers_encode_109,transfomers_encode_110,transfomers_encode_111,transfomers_encode_112,transfomers_encode_113,transfomers_encode_114,transfomers_encode_115,transfomers_encode_116,transfomers_encode_117,transfomers_encode_118,transfomers_encode_119,transfomers_encode_120,transfomers_encode_121,transfomers_encode_122,transfomers_encode_123,transfomers_encode_124,transfomers_encode_125,transfomers_encode_126,transfomers_encode_127,transfomers_encode_128,transfomers_encode_129,transfomers_encode_130,transfomers_encode_131,transfomers_encode_132,transfomers_encode_133,transfomers_encode_134,transfomers_encode_135,transfomers_encode_136,transfomers_encode_137,transfomers_encode_138,transfomers_encode_139,transfomers_encode_140,transfomers_encode_141,transfomers_encode_142,transfomers_encode_143,transfomers_encode_144,transfomers_encode_145,transfomers_encode_146,transfomers_encode_147,transfomers_encode_148,transfomers_encode_149,transfomers_encode_150,transfomers_encode_151,transfomers_encode_152,transfomers_encode_153,transfomers_encode_154,transfomers_encode

In [19]:
df_test = test.X

In [20]:
df = df[df['CO2/N2_selectivity']<172]
df = df[df['volume [A^3]']<78969]
df = df[df['weight [u]']<21348]
df = df[df['surface_area [m^2/g]']<6956]
df = df[df['void_volume [cm^3/g]']<6.1666]
df = df[df['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]']<11.02]
df = df[df['organic_linker1'] != 59]
df = df[df['organic_linker2'] != 59]
df = df.reset_index(drop = True)
len(df)

54056

In [21]:
df.head()

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],_space_group_crystal_system,_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,_cell_angle_gamma,mof_convex_hull_area,mof_convex_hull_volume,mof_convex_hull_npoints,mof_convex_hull_nsimplex,partial_charge_median,bond_type_countS,bond_type_countA,bond_type_countD,bond_type_countT,bond_type_countAm,transfomers_encode_0,transfomers_encode_1,transfomers_encode_2,transfomers_encode_3,transfomers_encode_4,transfomers_encode_5,transfomers_encode_6,transfomers_encode_7,transfomers_encode_8,transfomers_encode_9,transfomers_encode_10,transfomers_encode_11,transfomers_encode_12,transfomers_encode_13,transfomers_encode_14,transfomers_encode_15,transfomers_encode_16,transfomers_encode_17,transfomers_encode_18,transfomers_encode_19,transfomers_encode_20,transfomers_encode_21,transfomers_encode_22,transfomers_encode_23,transfomers_encode_24,transfomers_encode_25,transfomers_encode_26,transfomers_encode_27,transfomers_encode_28,transfomers_encode_29,transfomers_encode_30,transfomers_encode_31,transfomers_encode_32,transfomers_encode_33,transfomers_encode_34,transfomers_encode_35,transfomers_encode_36,transfomers_encode_37,transfomers_encode_38,transfomers_encode_39,transfomers_encode_40,transfomers_encode_41,transfomers_encode_42,transfomers_encode_43,transfomers_encode_44,transfomers_encode_45,transfomers_encode_46,transfomers_encode_47,transfomers_encode_48,transfomers_encode_49,transfomers_encode_50,transfomers_encode_51,transfomers_encode_52,transfomers_encode_53,transfomers_encode_54,transfomers_encode_55,transfomers_encode_56,transfomers_encode_57,transfomers_encode_58,transfomers_encode_59,transfomers_encode_60,transfomers_encode_61,transfomers_encode_62,transfomers_encode_63,transfomers_encode_64,transfomers_encode_65,transfomers_encode_66,transfomers_encode_67,transfomers_encode_68,transfomers_encode_69,transfomers_encode_70,transfomers_encode_71,transfomers_encode_72,transfomers_encode_73,transfomers_encode_74,transfomers_encode_75,transfomers_encode_76,transfomers_encode_77,transfomers_encode_78,transfomers_encode_79,transfomers_encode_80,transfomers_encode_81,transfomers_encode_82,transfomers_encode_83,transfomers_encode_84,transfomers_encode_85,transfomers_encode_86,transfomers_encode_87,transfomers_encode_88,transfomers_encode_89,transfomers_encode_90,transfomers_encode_91,transfomers_encode_92,transfomers_encode_93,transfomers_encode_94,transfomers_encode_95,transfomers_encode_96,transfomers_encode_97,transfomers_encode_98,transfomers_encode_99,transfomers_encode_100,transfomers_encode_101,transfomers_encode_102,transfomers_encode_103,transfomers_encode_104,transfomers_encode_105,transfomers_encode_106,transfomers_encode_107,transfomers_encode_108,transfomers_encode_109,transfomers_encode_110,transfomers_encode_111,transfomers_encode_112,transfomers_encode_113,transfomers_encode_114,transfomers_encode_115,transfomers_encode_116,transfomers_encode_117,transfomers_encode_118,transfomers_encode_119,transfomers_encode_120,transfomers_encode_121,transfomers_encode_122,transfomers_encode_123,transfomers_encode_124,transfomers_encode_125,transfomers_encode_126,transfomers_encode_127,transfomers_encode_128,transfomers_encode_129,transfomers_encode_130,transfomers_encode_131,transfomers_encode_132,transfomers_encode_133,transfomers_encode_134,transfomers_encode_135,transfomers_encode_136,transfomers_encode_137,transfomers_encode_138,transfomers_encode_139,transfomers_encode_140,transfomers_encode_141,transfomers_encode_142,transfomers_encode_143,transfomers_encode_144,transfomers_encode_145,transfomers_encode_146,transfomers_encode_147,transfomers_encode_148,transfomers_encode_149,transfomers_encode_150,transfomers_encode_151,transfomers_encode_152,transfomers_encode_153,transfomers_encode_154,transfomers_encode

In [22]:
df.insert(
    loc=3,
    column="density [g/cm^3]",
    value=(df["weight [u]"] / df["volume [A^3]"]) * 1.66054,
)

df_test.insert(
    loc=3,
    column="density [g/cm^3]",
    value=(df_test["weight [u]"] / df_test["volume [A^3]"]) * 1.66054,
)

In [23]:
#Add volumetric suface area
df['volume_surface_area'] = df['surface_area [m^2/g]']*df['density [g/cm^3]']
#df_pretest['volume_surface_area'] = df_pretest['surface_area [m^2/g]']*df_pretest['density [g/cm^3]']
df_test['volume_surface_area'] = df_test['surface_area [m^2/g]']*df_test['density [g/cm^3]']

In [24]:
from sklearn.model_selection import StratifiedKFold, KFold

In [25]:
kf = KFold()
for k, (train_index, test_index) in enumerate(kf.split(df)):
    print(f"K Fold: {k + 1}")
    print("TRAIN:", train_index, "TEST:", test_index)

K Fold: 1
TRAIN: [10812 10813 10814 ... 54053 54054 54055] TEST: [    0     1     2 ... 10809 10810 10811]
K Fold: 2
TRAIN: [    0     1     2 ... 54053 54054 54055] TEST: [10812 10813 10814 ... 21620 21621 21622]
K Fold: 3
TRAIN: [    0     1     2 ... 54053 54054 54055] TEST: [21623 21624 21625 ... 32431 32432 32433]
K Fold: 4
TRAIN: [    0     1     2 ... 54053 54054 54055] TEST: [32434 32435 32436 ... 43242 43243 43244]
K Fold: 5
TRAIN: [    0     1     2 ... 43242 43243 43244] TEST: [43245 43246 43247 ... 54053 54054 54055]


In [26]:
df.iloc[train_index]['topology'].value_counts()/len(df.iloc[train_index]['topology'])

pcu    0.668540
sra    0.122928
acs    0.081929
bcu    0.052862
etb    0.045208
fof    0.014614
nbo    0.009481
pts    0.003145
the    0.000832
tbo    0.000393
rht    0.000069
Name: topology, dtype: float64

In [27]:
df['topology'].value_counts()/len(df['topology'])

pcu    0.652416
sra    0.111754
acs    0.072924
nbo    0.055165
bcu    0.047599
etb    0.042530
fof    0.012968
pts    0.003459
the    0.000684
tbo    0.000444
rht    0.000055
Name: topology, dtype: float64

In [28]:
kfold = StratifiedKFold(random_state = 123, shuffle = True)
X = df.drop('topology', axis = 1)
y = df['topology']
kfold.get_n_splits(X, y)

5

In [29]:
i = 0
train_data = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
test_data = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
train_index = list()
test_index = []
for train_index, test_index in kfold.split(X,y):
    train_data[i] = df.iloc[train_index]
    test_data[i] = df.iloc[test_index]
    i+=1                

In [30]:
train_data[0].index

Int64Index([    0,     1,     2,     3,     4,     5,     7,     9,    10,
               11,
            ...
            54044, 54046, 54048, 54049, 54050, 54051, 54052, 54053, 54054,
            54055],
           dtype='int64', length=43244)

In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
'''
df['functional_groups'] = df['functional_groups'].fillna('NO_values')
#df_pretest['functional_groups'] = df_pretest['functional_groups'].fillna('NO_values')
df_test['functional_groups'] = df_test['functional_groups'].fillna('NO_values')

stat = pd.read_csv('functionalgroup_stat2.csv')
df['functional_groups'] = np.where(pd.isna(df['functional_groups']),'NO_values', df['functional_groups'])
df = df.merge(stat, on=['functional_groups','topology'], how = 'left')

df_test['functional_groups'] = np.where(pd.isna(df_test['functional_groups']),'NO_values', df_test['functional_groups'])
df_test = df_test.merge(stat, on=['functional_groups','topology'], how = 'left')
'''
df_processed = df
#df_processed_pretest = df_pretest.drop('MOFname', axis = 1)
df_processed_test = df_test

df_processed['topology'] = le.fit_transform(df_processed['topology'].values)
#df_processed_pretest['topology'] = le.transform(df_processed_pretest['topology'].values)
df_processed_test['topology'] = le.transform(df_processed_test['topology'].values)

#df_processed['functional_groups'] = le.fit_transform(df_processed['functional_groups'].values)
#df_processed_pretest['functional_groups'] = le.transform(df_processed_pretest['functional_groups'].values)
#df_processed_test['functional_groups'] = le.transform(df_processed_test['functional_groups'].values)

df_processed['_space_group_crystal_system'] = le.fit_transform(df_processed['_space_group_crystal_system'].values)
#df_processed_pretest['_space_group_crystal_system'] = le.fit_transform(df_processed_pretest['_space_group_crystal_system'].values)
df_processed_test['_space_group_crystal_system'] = le.fit_transform(df_processed_test['_space_group_crystal_system'].values)

#df_processed = df_processed[df_processed['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]']!=np.inf]
df_processed.shape, df_processed_test.shape#, df_processed_pretest.shape

((54056, 540), (17000, 539))

In [32]:
df_processed["metal_encoded"] = df_processed.groupby("metal_linker")["CO2_working_capacity [mL/g]"].transform("median")
df_processed["organic1_encoded"] = df_processed.groupby("organic_linker1")["CO2_working_capacity [mL/g]"].transform("median")
df_processed["organic2_encoded"] = df_processed.groupby("organic_linker2")["CO2_working_capacity [mL/g]"].transform("median")
df_processed["topology_encoded"] = df_processed.groupby("topology")["CO2_working_capacity [mL/g]"].transform("median")
df_processed["_space_group_crystal_system_encoded"] = df_processed.groupby("_space_group_crystal_system")["CO2_working_capacity [mL/g]"].transform("median")
#df_processed["functional_groups_encoded"] = df_processed.groupby("functional_groups")["CO2_working_capacity [mL/g]"].transform("median")

In [33]:
di = df_processed[["metal_linker","metal_encoded"]].drop_duplicates().set_index('metal_linker').to_dict()['metal_encoded']
df_processed= df_processed.replace({"metal_linker":di})
#df_processed_pretest= df_processed_pretest.replace({"metal_linker":di})
df_processed_test= df_processed_test.replace({"metal_linker":di})

di = df_processed[["organic_linker1","organic1_encoded"]].drop_duplicates().set_index('organic_linker1').to_dict()['organic1_encoded']
df_processed= df_processed.replace({"organic_linker1":di})
#df_processed_pretest= df_processed_pretest.replace({"organic_linker1":di})
df_processed_test= df_processed_test.replace({"organic_linker1":di})

di = df_processed[["organic_linker2","organic2_encoded"]].drop_duplicates().set_index('organic_linker2').to_dict()['organic2_encoded']
df_processed= df_processed.replace({"organic_linker2":di})
#df_processed_pretest= df_processed_pretest.replace({"organic_linker2":di})
df_processed_test= df_processed_test.replace({"organic_linker2":di})

di = df_processed[["topology","topology_encoded"]].drop_duplicates().set_index('topology').to_dict()['topology_encoded']
df_processed= df_processed.replace({"topology":di})
#df_processed_pretest= df_processed_pretest.replace({"topology":di})
df_processed_test= df_processed_test.replace({"topology":di})

di = df_processed[["_space_group_crystal_system","_space_group_crystal_system_encoded"]].drop_duplicates().set_index('_space_group_crystal_system').to_dict()['_space_group_crystal_system_encoded']
df_processed= df_processed.replace({"_space_group_crystal_system":di})
#df_processed_pretest = df_processed_pretest.replace({"_space_group_crystal_system":di})
df_processed_test = df_processed_test.replace({"_space_group_crystal_system":di})

#di = df_processed[["functional_groups","functional_groups_encoded"]].drop_duplicates().set_index('functional_groups').to_dict()['functional_groups_encoded']
#df_processed= df_processed.replace({"functional_groups":di})
#df_processed_pretest = df_processed_pretest.replace({"functional_groups":di})
#df_processed_test = df_processed_test.replace({"functional_groups":di})

df_processed = df_processed.drop(['metal_encoded','organic1_encoded','organic2_encoded',
                                  'topology_encoded',
                                  '_space_group_crystal_system_encoded',
                                  #'functional_groups_encoded'
                                 ],
                                axis = 1)

In [34]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

df_filled = imputer.fit_transform(df_processed.drop(['CO2_working_capacity [mL/g]'], axis = 1))
df_filled = pd.DataFrame(df_filled, columns = df_processed.drop('CO2_working_capacity [mL/g]', axis = 1).columns)
df_filled = pd.concat([df_filled, df_processed['CO2_working_capacity [mL/g]']], axis = 1)
df_filled.info()
'''
df_filled_pretest = imputer.transform(df_processed_pretest.drop(['Unnamed: 0'], axis = 1))
df_filled_pretest = pd.DataFrame(df_filled_pretest, columns = df_processed_pretest.columns)
df_filled_pretest.info()
'''
df_filled_test = imputer.transform(df_processed_test)
df_filled_test = pd.DataFrame(df_filled_test, columns = df_processed_test.columns)
df_filled_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54056 entries, 0 to 54055
Columns: 540 entries, volume [A^3] to CO2_working_capacity [mL/g]
dtypes: float64(540)
memory usage: 222.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Columns: 539 entries, volume [A^3] to volume_surface_area
dtypes: float64(539)
memory usage: 69.9 MB


In [47]:
# #'''
# mask = [False,  True,  True,  True,  True,  True,  True,  True,  True,
#         True,  True,  True, False,  True,  True,  True,  True,  True,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False,  True,  True,
#         True,  True,  True,  True,  True,  True, False, False, False,
#        False, False,  True,  True,  True,  True,  True,  True, False,
#        False, False,  True, False, False, False, False, False, False,
#        False, False, False, False,  True,  True,  True, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False,  True,  True,  True,  True,  True,  True,  True,
#         True,  True,  True,  True, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False,  True, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False,  True,  True,  True,  True,  True,  True,  True,  True,
#         True, False, False, False, False,  True,  True,  True,  True,
#         True,  True,  True, False, False, False, False, False,  True,
#         True,  True, False, False, False,  True,  True,  True, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False,  True,
#         True, False,  True,  True, False, False, False, False, False,
#         True,  True,  True,  True, False, False, False, False,  True,
#        False, False, False, False,  True, False, False, False,  True,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False,  True,  True, False, False, False,
#        False, False, False, False, False, False, False,  True, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False,  True,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False, False, False,
#        False, False, False, False, False, False, False,  True]
# X_masked = df_filled.drop('CO2_working_capacity [mL/g]', axis = 1).iloc[:,mask]
# print(X_masked.shape)
# X_masked.head()
# #'''

(54056, 84)


,weight [u],surface_area [m^2/g],density [g/cm^3],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,topology,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],_cell_length_a,_cell_length_b,_cell_length_c,_cell_angle_alpha,_cell_angle_beta,RDF_electronegativity_2.07,RDF_electronegativity_2.09,RDF_electronegativity_2.12,RDF_electronegativity_2.16,RDF_electronegativity_2.20,RDF_electronegativity_2.24,RDF_electronegativity_2.29,RDF_electronegativity_2.35,RDF_electronegativity_2.76,RDF_electronegativity_2.84,RDF_electronegativity_2.93,RDF_electronegativity_3.02,RDF_electronegativity_3.12,RDF_electronegativity_3.22,RDF_electronegativity_3.67,RDF_electronegativity_5.28,RDF_electronegativity_5.45,RDF_electronegativity_5.63,RDF_hardness_2.04,RDF_hardness_2.07,RDF_hardness_2.09,RDF_hardness_2.12,RDF_hardness_2.16,RDF_hardness_2.20,RDF_hardness_2.24,RDF_hardness_2.29,RDF_hardness_2.35,RDF_hardness_2.40,RDF_hardness_2.46,RDF_hardness_13.63,RDF_vdWaalsVolume_2.12,RDF_vdWaalsVolume_2.16,RDF_vdWaalsVolume_2.20,RDF_vdWaalsVolume_2.24,RDF_vdWaalsVolume_2.29,RDF_vdWaalsVolume_2.35,RDF_vdWaalsVolume_2.40,RDF_vdWaalsVolume_2.46,RDF_vdWaalsVolume_2.53,RDF_vdWaalsVolume_2.93,RDF_vdWaalsVolume_3.02,RDF_vdWaalsVolume_3.12,RDF_vdWaalsVolume_3.22,RDF_vdWaalsVolume_3.33,RDF_vdWaalsVolume_3.44,RDF_vdWaalsVolume_3.55,RDF_vdWaalsVolume_4.34,RDF_vdWaalsVolume_4.48,RDF_vdWaalsVolume_4.63,RDF_vdWaalsVolume_5.28,RDF_vdWaalsVolume_5.45,RDF_vdWaalsVolume_5.63,label_topology_pcu,label_topology_sra,label_topology_etb,label_topology_bcu,metal_linker_4,metal_linker_9,metal_linker_10,metal_linker_12,organic_linker1_5.0,organic_linker1_10.0,organic_linker1_14.0,organic_linker2_17.0,organic_linker2_18.0,organic_linker2_29.0,density,volume_surface_area
0,2211.697211,603.61,1.326090,0.13794,0.104020,82.408689,80.981272,86.117464,82.408689,33.616780,7.147286,8.463295,17.684225,18.960098,100.063446,91.815775,0.001221,0.001286,0.001366,0.001462,0.001574,0.001703,0.001846,0.001999,0.002693,0.002705,0.002689,0.002667,0.002677,0.002758,0.003848,0.006521,0.006521,0.006557,0.037334,0.038670,0.040333,0.042350,0.044748,0.047550,0.050764,0.054377,0.058329,0.062498,0.066690,2.000000e-12,0.000482,0.000538,0.000607,0.000688,0.000780,0.000876,0.000969,0.001052,0.001118,0.001177,0.001131,0.001097,0.001114,0.001211,0.001389,0.001608,0.002493,0.002610,0.002762,0.003040,0.002992,0.002951,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.326090,800.441258
1,773.687960,788.50,1.178856,0.14874,0.126173,88.457259,87.988359,99.723238,87.450622,19.263726,6.347967,10.732110,9.552271,10.631996,89.202223,89.943258,0.001483,0.001588,0.001712,0.001852,0.002006,0.002166,0.002325,0.002476,0.003023,0.003099,0.003202,0.003342,0.003514,0.003692,0.003863,0.006422,0.006278,0.006159,0.045475,0.048142,0.051396,0.055207,0.059488,0.064089,0.068800,0.073373,0.077570,0.081215,0.084236,0.000000e+00,0.000632,0.000692,0.000761,0.000837,0.000916,0.000995,0.001068,0.001130,0.001175,0.001210,0.001228,0.001262,0.001310,0.001362,0.001398,0.001404,0.001851,0.002055,0.002259,0.002406,0.002359,0.002318,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.178856,929.528288
2,1304.638720,1441.53,0.982408,0.21814,0.222046,134.445769,85.075748,99.723238,134.445769,25.701377,6.190085,6.935530,17.504896,19.274980,109.537419,90.234580,0.001025,0.001120,0.001234,0.001367,0.001517,0.001677,0.001840,0.001995,0.002362,0.002266,0.002122,0.001965,0.001843,0.001789,0.002254,0.002941,0.002696,0.002729,0.027127,0.029272,0.031953,0.035178,0.038911,0.043055,0.047434,0.051796,0.055835,0.059249,0.061811,0.000000e+00,0.000777,0.000847,0.000927,0.001012,0.001097,0.001176,0.001240,0.001283,0.001300,0.001052,0.000969,0.000908,0.000892,0.000936,0.001037,0.001159,0.001140,0.001129,0.001276,0.001408,0.001356,0.001439,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.982408,1416.170818
3,1543.027680,2430.55,0.647909,0.37094,0.572519,134.445769,67.618656,78.874813,134.445769,17.146541,5.

In [35]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
X_train = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
y_train = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
X_test = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
y_test = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]

for i in range(5):
    X_train[i] = df_filled.iloc[train_data[i].index.tolist(),].drop('CO2_working_capacity [mL/g]', axis = 1)
    #X_train[i] = X_masked.iloc[train_data[i].index.tolist(),]
    y_train[i] = df_filled.iloc[train_data[i].index.tolist(),]['CO2_working_capacity [mL/g]']
    X_test[i] = df_filled.iloc[test_data[i].index.tolist(),].drop('CO2_working_capacity [mL/g]', axis = 1)
    #X_test[i] = X_masked.iloc[test_data[i].index.tolist(),]
    y_test[i] = df_filled.iloc[test_data[i].index.tolist(),]['CO2_working_capacity [mL/g]']    
    scaler = MinMaxScaler().fit(X_train[i])
    scaler.transform(X_train[i])
    scaler.transform(X_test[i])

In [37]:
for i in range(5):
    print(X_train[i].shape, y_train[i].shape, X_test[i].shape, y_test[i].shape)

(43244, 539) (43244,) (10812, 539) (10812,)
(43245, 539) (43245,) (10811, 539) (10811,)
(43245, 539) (43245,) (10811, 539) (10811,)
(43245, 539) (43245,) (10811, 539) (10811,)
(43245, 539) (43245,) (10811, 539) (10811,)


In [51]:
'''
from boruta import BorutaPy
from sklearn.ensemble import ExtraTreesRegressor
model_cv = ExtraTreesRegressor(random_state=42)
feat_selector = BorutaPy(model_cv, n_estimators='auto', verbose=2, random_state=1)
#feat_selector.fit(X_train[0].iloc[0:20000].values, y_train[0].iloc[0:20000].values)
feat_selector.fit(X_train_final[0:10000].values, y_train_final[0:10000].values)
feat_selector.support_
'''

"\nfrom boruta import BorutaPy\nfrom sklearn.ensemble import ExtraTreesRegressor\nmodel_cv = ExtraTreesRegressor(random_state=42)\nfeat_selector = BorutaPy(model_cv, n_estimators='auto', verbose=2, random_state=1)\n#feat_selector.fit(X_train[0].iloc[0:20000].values, y_train[0].iloc[0:20000].values)\nfeat_selector.fit(X_train_final[0:10000].values, y_train_final[0:10000].values)\nfeat_selector.support_\n"

In [87]:
# check ranking of features
#feat_selector.ranking_

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV, ElasticNet
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
import datetime
from dateutil.relativedelta import *
import pandas as pd
import numpy as np

from AutoML2 import AutoML2

import seaborn as sns
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, LinearRegression, LassoLarsCV, BayesianRidge, SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [39]:
#Rename columns
for i in range(5):
    X_train[i].columns = range(len(X_train[i].columns))
    X_test[i].columns = range(len(X_test[i].columns))

In [40]:
def model_fit_predict_mae(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    mae = mean_absolute_error(y_pred, y_test)
    mae_train = mean_absolute_error(y_pred_train, y_train)
    return (mae, mae_train)

def cv_mae(model, k, X_train, y_train, X_test, y_test):
    mae = []
    mae_train = []
    for i in range(k):
        temp_mae, temp_mae_train = model_fit_predict_mae(model, X_train[i], y_train[i], X_test[i], y_test[i])
        mae.append(temp_mae)
        mae_train.append(temp_mae_train)
    return (np.log10(mae), np.log10(mae_train))
scores = {}
scores2 = {}
scores_train = {}
scores2_train = {}
k = 5

In [ ]:
estimators = [
    ('catboost', CatBoostRegressor(
                    iterations=2500,
                    grow_policy='Lossguide',
                    verbose=False,
                    #task_type='GPU',
                    l2_leaf_reg=70,
                  #  depth = 11,
                    learning_rate=0.1
                )), 
    ('ext', ExtraTreesRegressor()),
    ('lightgbm', LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=1000
                )),
    ('xgb', XGBRegressor(
                    grow_policy='lossguide',
                    tree_method='hist',
                    n_estimators=2500,
                    eta=0.1,
                    max_depth=14,
                    reg_lambda=10.5
                ))
]
score, score_train = cv_mae(VotingRegressor(estimators=estimators), k, X_train, y_train, X_test, y_test)
print("VotingRegressor_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print("VotingRegressor_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores['VotingRegressor'] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train['VotingRegressor'] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2['VotingRegressor'] = pd.Series(score)
scores2_train['VotingRegressor'] = pd.Series(score_train)



In [62]:
model_name = "Catboost"
score, score_train = cv_mae(CatBoostRegressor(
                    iterations=2500,
                    grow_policy='Lossguide',
                    verbose=False,
                   # task_type='GPU',
                    l2_leaf_reg=50,
                    learning_rate=0.1
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

Catboost_train: 0.9899 (0.0015)
Catboost_test: 1.2351 (0.0040)
[0.99166533 0.98922187 0.98918338 0.99135034 0.98816408] [1.24197643 1.23502881 1.23191823 1.23251212 1.23414478]


In [63]:
model_name = "Catboost"
score, score_train = cv_mae(CatBoostRegressor(
                    iterations=2500,
                    grow_policy='Lossguide',
                    verbose=False,
                   # task_type='GPU',
                    l2_leaf_reg=30,
                    learning_rate=0.1,
                   # depth = 9
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

Catboost_train: 0.9527 (0.0019)
Catboost_test: 1.2360 (0.0027)
[0.95119487 0.95179546 0.95194824 0.95587039 0.95273375] [1.24003098 1.23711286 1.23314238 1.23465976 1.23501546]


In [64]:
model_name = "Catboost"
score, score_train = cv_mae(CatBoostRegressor(
                    iterations=2500,
                    grow_policy='Lossguide',
                    verbose=False,
                   # task_type='GPU',
                    l2_leaf_reg=70,
                  #  depth = 11,
                    learning_rate=0.1
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

Catboost_train: 1.0145 (0.0023)
Catboost_test: 1.2343 (0.0035)
[1.01553769 1.01175238 1.01426495 1.01337121 1.01777423] [1.23872522 1.23669252 1.23075787 1.23082629 1.23461004]


In [65]:
model_name = "LightGBM"
score, score_train = cv_mae(LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=1200
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

LightGBM_train: 1.0483 (0.0010)
LightGBM_test: 1.2352 (0.0038)
[1.04828273 1.04670274 1.04819041 1.04857058 1.04952452] [1.24057616 1.23646742 1.23030061 1.23500147 1.23356244]


In [66]:
model_name = "LightGBM"
score, score_train = cv_mae(LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=800
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

LightGBM_train: 1.0239 (0.0017)
LightGBM_test: 1.2346 (0.0038)
[1.02247087 1.02324271 1.02467    1.02249739 1.02650458] [1.24111369 1.23509863 1.23197319 1.23226743 1.23272251]


In [67]:
model_name = "LightGBM"
score, score_train = cv_mae(LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=1000
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

LightGBM_train: 1.0368 (0.0012)
LightGBM_test: 1.2350 (0.0047)
[1.03590977 1.03559754 1.03722771 1.03680144 1.03852001] [1.24288163 1.23473061 1.23112224 1.23415652 1.23193539]


In [68]:
model_name = "LightGBM"
score, score_train = cv_mae(LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=1200
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

LightGBM_train: 1.0483 (0.0010)
LightGBM_test: 1.2352 (0.0038)
[1.04828273 1.04670274 1.04819041 1.04857058 1.04952452] [1.24057616 1.23646742 1.23030061 1.23500147 1.23356244]


In [69]:
model_name = "XGBoost"
score, score_train = cv_mae(XGBRegressor(
                    #grow_policy='lossguide',
                    n_estimators=500,
                    eta=0.1,
                    #max_depth=14,
                    reg_lambda=10.5
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

XGBoost_train: 1.0482 (0.0019)
XGBoost_test: 1.2356 (0.0038)
[1.04770926 1.04544629 1.05046524 1.04813307 1.04909987] [1.24143504 1.23591647 1.23147091 1.23294751 1.23620886]


In [70]:
model_name = "XGBoost"
score, score_train = cv_mae(XGBRegressor(
                    #grow_policy='lossguide',
                    n_estimators=500,
                    eta=0.1,
                    #max_depth=14,
                    reg_lambda=100
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

XGBoost_train: 1.1019 (0.0030)
XGBoost_test: 1.2352 (0.0035)
[1.09979286 1.09820075 1.10402065 1.1020561  1.10560633] [1.24105306 1.23475965 1.23511564 1.23269829 1.23231884]


In [71]:
model_name = "XGBoost"
score, score_train = cv_mae(XGBRegressor(
                    #grow_policy='lossguide',
                    n_estimators=500,
                    eta=0.1,
                    #max_depth=14,
                    reg_lambda=1
                ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

XGBoost_train: 1.0392 (0.0018)
XGBoost_test: 1.2379 (0.0043)
[1.03833153 1.04010545 1.03652876 1.04124487 1.0396782 ] [1.2451428  1.23722719 1.23492274 1.23442763 1.23793065]


In [87]:
model_name = "ExtraTrees"
score, score_train = cv_mae(ExtraTreesRegressor(), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

ExtraTrees_train: -12.8770 (0.0008)
ExtraTrees_test: 1.2531 (0.0045)
[-12.87795576 -12.87696783 -12.87590386 -12.87747109 -12.87663242] [1.2605109  1.25350489 1.24870875 1.2510415  1.25154965]


In [88]:
model_name = "AdaBoost"
score, score_train = cv_mae(AdaBoostRegressor(), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

AdaBoost_train: 1.6244 (0.0117)
AdaBoost_test: 1.6258 (0.0124)
[1.61865461 1.62900699 1.61154552 1.620569   1.64215409] [1.62116184 1.6366078  1.61101508 1.61974969 1.64061483]


In [111]:
model_name = "AdaBoost_LGBM"
score, score_train = cv_mae(AdaBoostRegressor(LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=1200
                )
                                             ), k, X_train, y_train, X_test, y_test)
print(model_name + "_train: {:.4f} ({:.4f})".format(pd.Series(score_train).mean(), pd.Series(score_train).std()))
print(model_name + "_test: {:.4f} ({:.4f})".format(pd.Series(score).mean(), pd.Series(score).std()))
scores[model_name] = (pd.Series(score).mean(), pd.Series(score).std())
scores_train[model_name] = (pd.Series(score_train).mean(), pd.Series(score_train).std())
print(score_train, score)
scores2[model_name] = pd.Series(score)
scores2_train[model_name] = pd.Series(score_train)

AdaBoost_LGBM_train: 1.0431 (0.0018)
AdaBoost_LGBM_test: 1.2415 (0.0034)
[1.04021732 1.04500642 1.04407354 1.04280472 1.04329022] [1.2473018  1.23966641 1.24102482 1.24094741 1.23865369]


In [41]:
X_train_final = df_filled.drop('CO2_working_capacity [mL/g]', axis = 1)
#X_train_final = X_masked
y_train_final = df_filled['CO2_working_capacity [mL/g]']
X_submission = df_filled_test
#X_submission = df_filled_test.iloc[:,mask]
scaler = MinMaxScaler().fit(X_train_final)
scaler.transform(X_train_final)
scaler.transform(X_submission)

array([[0.00833814, 0.01730468, 0.07939607, ..., 0.        , 0.        ,
        0.2075748 ],
       [0.04877457, 0.15905037, 0.11783521, ..., 0.        , 0.        ,
        0.42770403],
       [0.01379569, 0.01486663, 0.45073446, ..., 0.        , 0.        ,
        0.8290966 ],
       ...,
       [0.05630434, 0.04261945, 0.61449717, ..., 0.        , 0.        ,
        0.6814745 ],
       [0.05647501, 0.03737934, 0.62072664, ..., 0.        , 0.        ,
        0.6290715 ],
       [0.34775716, 0.21020521, 0.71821996, ..., 0.        , 0.        ,
        0.52070218]])

In [42]:
#Rename columns
X_train_final.columns = range(len(X_train_final.columns))
X_submission.columns = range(len(X_submission.columns))

In [60]:
from sklearn.ensemble import BaggingRegressor
model_bag = BaggingRegressor(base_estimator=XGBRegressor(
                    grow_policy='lossguide',
                    tree_method='hist',
                    n_estimators=2500,
                    eta=0.1,
                    max_depth=14,
                    reg_lambda=10.5
                ), n_estimators=5, max_features = 100, max_samples = 10000, random_state=0)
estimators = [
    ('catboost', CatBoostRegressor(
                    iterations=2500,
                    grow_policy='Lossguide',
                    verbose=False,
                    #task_type='GPU',
                    l2_leaf_reg=70,
                  #  depth = 11,
                    learning_rate=0.1
                )), 
    ('ext', ExtraTreesRegressor()),
    ('lightgbm', LGBMRegressor(
                    num_iterations=2500,
                    #device="gpu",
                    learning_rate=0.1,
                    reg_lambda=1000
                )),
    #('xgb', model_bag)
]


model = VotingRegressor(estimators=estimators, weights = [0.4,0.2,0.4])
model.fit(X_train_final, y_train_final)
y_pred = model.predict(X_submission)

In [61]:
y_pred = model.predict(X_submission)

In [62]:
pd.DataFrame(y_pred).median()

0    91.305274
dtype: float64

In [63]:
pd.DataFrame(np.abs(y_pred)).mean()

0    109.209603
dtype: float64

In [64]:
index = pd.Series([str(i) for i in (np.arange(68614,85614))])
submission = pd.DataFrame([index, y_pred]).T
submission.columns = ['id','CO2_working_capacity [mL/g]']
submission = submission.set_index('id')
submission.to_csv('submission.csv')
submission

,CO2_working_capacity [mL/g]
id,
68614,181.929
68615,69.6461
68616,67.7321
68617,62.027
68618,60.5611
...,...
85609,-6.09849
85610,3.27881
85611,1.74873
